Telemetry analysis for unified-urlbar experiment.
https://bugzilla.mozilla.org/show_bug.cgi?id=1219505

In [247]:
import pandas as pd
import ujson as json
import numpy as np

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Populating the interactive namespace from numpy and matplotlib


Define the pings we care about.
The experiment ran on Firefox Beta 44 and 45, between Jan 11th and Feb 24th.
We care about "main" telemetry pings.

NOTE: For now, while we are developing the notebook, we only take a small fraction for a single day.

In [248]:
PING_OPTIONS = {
    "app": "Firefox",
    "channel": "beta",
    "version": ("44.0", "45.0"),
    "build_id": "*",
    "submission_date": "20160114",
    "fraction": 0.01
}
pings = get_pings(sc, doc_type="main", **PING_OPTIONS)

pings.count()

52488

We only need a subset of the ping data.

In [249]:
pings_data = get_pings_properties(pings,
                                  ["clientId",
                                   "environment/addons/activeExperiment/id",
                                   "environment/addons/activeExperiment/branch",
                                   "environment/settings/defaultSearchEngine",
                                   "environment/settings/userPrefs/browser.urlbar.suggest.searches",
                                   "environment/settings/userPrefs/browser.urlbar.userMadeSearchSuggestionsChoice",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/defaultKept",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/searchbar",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/urlbar",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search-oneoff",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/urlbar/search-settings",
                                   "payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings",
                                   "payload/histograms/FX_URLBAR_SELECTED_RESULT_TYPE"])

To prevent pseudoreplication, let's consider only a single submission for each client. As this step requires a distributed shuffle, it should always be run only after extracting the attributes of interest with get_pings_properties.

In [250]:
pings_data = get_one_ping_per_client(pings_data)
pings_data.count()

38187

Only consider pings from users having the experiment.
Also discard users not having a default urlbar.

In [251]:
def experiment_filter(d):
    toolbar = d["payload/simpleMeasurements/UITelemetry/toolbars/defaultKept"]
    try:
        return d["environment/addons/activeExperiment/id"] == "unified-urlbar@experiments.mozilla.org" \
            and d["environment/addons/activeExperiment/branch"] in ("control", "unified", "customized") \
            and toolbar is not None and "urlbar-container" in toolbar
    except KeyError:
        raise ValueError("Whoa nellie, missing a key: " + repr(d))

experiment_data = pings_data.filter(experiment_filter).cache()

How many pings are left?

In [252]:
experiment_data.count()

3296

Normalize number of samples.

In [253]:
c = experiment_data.map(lambda d: (d["environment/addons/activeExperiment/branch"])).countByValue()
normalized_c = min(c["control"], c["unified"])
control_data = sc.parallelize(experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "control").take(normalized_c))
unified_data = sc.parallelize(experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "unified").take(normalized_c))
customized_data = experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "customized")

Analyze data.

In [254]:
class Accumulator():
    def __init__(self, cx):
        # number of analyzed pings
        self.ping_count = cx.accumulator(0)
        # number of searches from the search bar
        self.searchbar_searches = cx.accumulator(0)
        # search coming from urlbar oneoff
        self.from_searchbar_oneoff = cx.accumulator(0)
        # number of searches from the urlbar
        self.urlbar_searches = cx.accumulator(0)
        # search coming from urlbar suggestion
        self.from_urlbar_suggestion = cx.accumulator(0)
        # search coming from urlbar oneoff
        self.from_urlbar_oneoff = cx.accumulator(0)
        # has urlbar suggestions enabled
        self.has_suggestions = cx.accumulator(0)
        # made urlbar suggestions choice
        self.made_choice = cx.accumulator(0)
        # disabled urlbar suggestions
        self.disabled_suggestions = cx.accumulator(0)
        # search bar on nav-bar
        self.has_searchbar = cx.accumulator(0)
        # did any search
        self.ping_has_searches = cx.accumulator(0)
        # total number of searches
        self.total_searches = cx.accumulator(0)
        # searchbar settings
        self.searchbar_settings = cx.accumulator(0)
        # urlbar settings
        self.urlbar_settings = cx.accumulator(0)

def process_data(acc, d):
    acc.ping_count.add(1)

    has_suggestions = d["environment/settings/userPrefs/browser.urlbar.suggest.searches"]
    if has_suggestions is True:
        acc.has_suggestions.add(1)

    made_choice = d["environment/settings/userPrefs/browser.urlbar.userMadeSearchSuggestionsChoice"]
    if made_choice is True:
        acc.made_choice.add(1)
        if has_suggestions is not True:
            acc.disabled_suggestions.add(1)

    ping_has_searches = False
    searchbar_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/searchbar"]
    if searchbar_searches is not None:
        acc.searchbar_searches.add(searchbar_searches)
        acc.total_searches.add(searchbar_searches)
        ping_has_searches = True

    urlbar_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/urlbar"]
    if urlbar_searches is not None:
        acc.urlbar_searches.add(urlbar_searches)
        acc.total_searches.add(urlbar_searches)
        ping_has_searches = True

    widgetsInDefaultPosition = d["payload/simpleMeasurements/UITelemetry/toolbars/defaultKept"]
    if widgetsInDefaultPosition is not None:
        if "search-container" in widgetsInDefaultPosition:
            acc.has_searchbar.add(1)

    urlbar_result = d["payload/histograms/FX_URLBAR_SELECTED_RESULT_TYPE"]
    if urlbar_result is not None:
        acc.from_urlbar_suggestion.add(int(urlbar_result[5]))

    oneoff = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search-oneoff"]
    if oneoff is not None:
        urlbar_oneoff = {k:v for (k,v) in oneoff.iteritems() if "urlbar" in k}.values()
        for o in urlbar_oneoff:
            oneoff = {k:v for (k,v) in o.iteritems() if "unknown" not in k}.values()
            for v in oneoff:
                acc.from_urlbar_oneoff.add(sum(v.values()))

    oneoff = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search-oneoff"]
    if oneoff is not None:
        searchbar_oneoff = {k:v for (k,v) in oneoff.iteritems() if "urlbar" not in k}.values()
        for o in searchbar_oneoff:
            oneoff = {k:v for (k,v) in o.iteritems() if "oneoff" in k}.values()
            for v in oneoff:
                acc.from_searchbar_oneoff.add(sum(v2.values()))
                acc.searchbar_searches.add(sum(v2.values()))
                acc.total_searches.add(sum(v2.values()))
                ping_has_searches = True
            other = {k:v for (k,v) in o.iteritems() if "oneoff" not in k}.values()
            for v in other:
                acc.searchbar_searches.add(sum(v.values()))
                acc.total_searches.add(sum(v.values()))
                ping_has_searches = True

    if ping_has_searches is True:
        acc.ping_has_searches.add(1)

    urlbar_settings = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings"]
    if urlbar_settings is not None:
        acc.urlbar_settings.add(urlbar_settings)

    searchbar_settings = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings"]
    if searchbar_settings is not None:
        acc.searchbar_settings.add(searchbar_settings)

control = Accumulator(sc)
control_data.foreach(lambda d: process_data(control, d))
unified = Accumulator(sc)
unified_data.foreach(lambda d: process_data(unified, d))
customized = Accumulator(sc)
customized_data.foreach(lambda d: process_data(customized, d))

Print results.

In [255]:
def pval(name, type, a1, a2, a3):
    print ("{:30s} {:>15" + type + "} {:>15" + type + "} {:>15" + type + "}").format(name, a1, a2, a3)

def pper(name, v, t, a1, a2, a3):
    def perc(acc):
          return (float(getattr(acc, v).value) / getattr(acc, t).value) * 100
    print ("{:30s} {:>14.1f}% {:>14.1f}% {:>14.1f}%").format(name, perc(a1), perc(a2), perc(a3))

pval("", "s", "control", "unified", "customized")
pval("number of users", "d", control.ping_count.value, unified.ping_count.value, customized.ping_count.value)
pval("from urlbar & searchbar", "d", control.total_searches.value, unified.total_searches.value, customized.total_searches.value)
pper("users who searched", "ping_has_searches", "ping_count", control, unified, customized)
print ""
#pval("from urlbar", "d", control.urlbar_searches.value, unified.urlbar_searches.value, customized.urlbar_searches.value)
pper("from urlbar", "urlbar_searches", "total_searches", control, unified, customized)
#pval("  from urlbar suggestion", "d", control.from_urlbar_suggestion.value, unified.from_urlbar_suggestion.value, customized.from_urlbar_suggestion.value)
pper("  from suggestion", "from_urlbar_suggestion", "urlbar_searches", control, unified, customized)
#pval("  from urlbar one-off", "d", control.from_urlbar_oneoff.value, unified.from_urlbar_oneoff.value, customized.from_urlbar_oneoff.value)
pper("  from one-off", "from_urlbar_oneoff", "urlbar_searches", control, unified, customized)
print ""
pper("from searchbar", "searchbar_searches", "total_searches", control, unified, customized)
#pval("  from searchbar one-off", "d", control.from_searchbar_oneoff.value, unified.from_searchbar_oneoff.value, customized.from_searchbar_oneoff.value)
pper("  from one-off", "from_searchbar_oneoff", "searchbar_searches", control, unified, customized)
print ""
pper("made suggestions choice", "made_choice", "ping_count", control, unified, customized)
pper("  enabled urlbar suggestions", "has_suggestions", "made_choice", control, unified, customized)
pper("  disabled urlbar suggestions", "disabled_suggestions", "made_choice", control, unified, customized)
print ""
pper("searchbar in nav-bar", "has_searchbar", "ping_count", control, unified, customized)
pper("searchbar settings", "searchbar_settings", "ping_count", control, unified, customized)
pper("urlbar settings", "urlbar_settings", "ping_count", control, unified, customized)

                                       control         unified      customized
number of users                           1576            1576              91
from urlbar & searchbar                   1191             938              52
users who searched                       16.3%           13.2%           12.1%

from urlbar                              27.0%           54.5%           67.3%
  from suggestion                         0.3%            0.6%            0.0%
  from one-off                            0.0%            0.8%            5.7%

from searchbar                           73.0%           45.5%           32.7%
  from one-off                            0.0%            0.0%            0.0%

made suggestions choice                  32.2%           30.8%           36.3%
  enabled urlbar suggestions             49.8%           52.2%           51.5%
  disabled urlbar suggestions            52.6%           50.7%           48.5%

searchbar in nav-bar                     99.9%  